In [ ]:
!pip install optuna

In [ ]:
!unzip classification_dataset.zip -d dataset

## Разделение данных и балансировка

In [ ]:
import os
import shutil

source_dir = '/content/dataset/classification_dataset'
destination_dir_bottom = 'pallet_bottom_dataset'
destination_dir_side = 'pallet_side_dataset'

os.makedirs(destination_dir_bottom, exist_ok=True)
os.makedirs(destination_dir_side, exist_ok=True)

def copy_class_files(source, dest):
    for file_name in os.listdir(source):
        src_file = os.path.join(source, file_name)
        dest_file = os.path.join(dest, file_name)
        shutil.copy2(src_file, dest_file)

bottom_classes = ['good_pallet', 'replace_pallet']
for class_name in bottom_classes:
    source_class_path = os.path.join(source_dir, 'pallet_bottom', class_name)
    dest_class_path = os.path.join(destination_dir_bottom, class_name)
    os.makedirs(dest_class_path, exist_ok=True)
    copy_class_files(source_class_path, dest_class_path)


side_classes = ['good_pallet', 'replace_pallet']
for class_name in side_classes:
    source_class_path = os.path.join(source_dir, 'pallet_side', class_name)
    dest_class_path = os.path.join(destination_dir_side, class_name)
    os.makedirs(dest_class_path, exist_ok=True)
    copy_class_files(source_class_path, dest_class_path)

print("Разделение датасета завершено.")


In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

dataset_dir = '/content/pallet_side_dataset'
class_folders = ['good_pallet', 'replace_pallet']

class_counts = {cls: len(os.listdir(os.path.join(dataset_dir, cls))) for cls in class_folders}

minority_class = min(class_counts, key=class_counts.get)
majority_class = max(class_counts, key=class_counts.get)

minority_count = class_counts[minority_class]
majority_count = class_counts[majority_class]

print(f"Миноритарная папка: {minority_class}, количество изображений: {minority_count}")
print(f"Мажоритарная папка: {majority_class}, количество изображений: {majority_count}")

num_to_add = majority_count - minority_count

minority_folder = os.path.join(dataset_dir, minority_class)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

images = os.listdir(minority_folder)
i = 0
while i < num_to_add:
    img_name = images[i % len(images)]
    img_path = os.path.join(minority_folder, img_name)

    img = load_img(img_path)
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)

    for batch in datagen.flow(img_array, batch_size=1, save_to_dir=minority_folder, save_prefix='aug', save_format='jpeg'):
        i += 1
        if i >= num_to_add:
            break

print(f"Теперь в папке '{minority_class}' {minority_count + num_to_add} изображений, как и в '{majority_class}'.")


## Обучение кастомной модели

In [ ]:
import os
import numpy as np
import optuna
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

data_dir = '/content/pallet_side_dataset'


img_size = (224, 224)

def create_data_generators(data_dir, img_size, batch_size):
    datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        validation_split=0.3,
        horizontal_flip=True,
        zoom_range=0.2,
        rotation_range=30,
        brightness_range=[0.7, 1.3]
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )

    return train_generator, val_generator

def build_mobilenetv2_model(input_shape, dropout_rate):
    inputs = Input(shape=input_shape)
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)

    base_model.trainable = False

    return model

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.8)
    batch_size = trial.suggest_categorical('batch_size', [2, 4, 8, 16, 32])
    epochs = trial.suggest_int('epochs', 10, 60)

    train_gen, val_gen = create_data_generators(data_dir, img_size, batch_size)

    model = build_mobilenetv2_model((img_size[0], img_size[1], 3), dropout_rate)
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, mode='min')

    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=[early_stopping],
        verbose=0
    )

    val_loss = min(history.history['val_loss'])

    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("Best hyperparameters: ", study.best_params)

best_params = study.best_params

train_gen, val_gen = create_data_generators(data_dir, img_size, best_params['batch_size'])
model = build_mobilenetv2_model((img_size[0], img_size[1], 3), best_params['dropout_rate'])
model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history = model.fit(
    train_gen,
    epochs=best_params['epochs'],
    validation_data=val_gen,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, mode='min')]
)

val_gen.reset()
val_pred_prob = model.predict(val_gen)
val_pred = (val_pred_prob > 0.5).astype(int)
val_true = val_gen.classes

print("\nFinal Classification Report:")
print(classification_report(val_true, val_pred, target_names=['good_pallet', 'replace_pallet']))


In [ ]:
import keras
keras.saving.save_model(model, 'pallet_side_classifier_mn_optuna.h5')

## Тестирование модели

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, **kwargs):
        super(FocalLoss, self).__init__(**kwargs)
        self.gamma = gamma
        self.alpha = alpha

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, dtype='float32')
        alpha_t = y_true * self.alpha + (tf.ones_like(y_true) - y_true) * (1 - self.alpha)
        p_t = y_true * y_pred + (tf.ones_like(y_true) - y_true) * (tf.ones_like(y_true) - y_pred) + tf.keras.backend.epsilon()
        focal_loss = -alpha_t * tf.pow((tf.ones_like(y_true) - p_t), self.gamma) * tf.math.log(p_t)
        return tf.reduce_mean(focal_loss)

model = tf.keras.models.load_model(
    '/content/pallet_side_classifier_mn_optuna.h5',
    custom_objects={'FocalLoss': FocalLoss}
)

def prepare_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

image_path = '/content/0_IMG_2863.jpg'
prepared_image = prepare_image(image_path)
predictions = model.predict(prepared_image)

print(f"Вероятность принадлежности к классу 'replace_pallet': {predictions[0][0]}")

predicted_class = (predictions > 0.5).astype(int)[0][0]
class_labels = ['good_pallet', 'replace_pallet']

print(f"Предсказанный класс: {class_labels[predicted_class]}")
